In [1]:
import pandas as pd
import numpy as np
from IPython.display import clear_output
from google.colab import drive
import matplotlib.pyplot as plt
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
currency_type = 'XRP'

In [3]:
### Code for LOB transactions datasets
for n in [0, 1]:
  LOB = []
  for k in range(n*6, (n+1)*6):
    data = pd.read_csv('/content/drive/MyDrive/Sequential_datasets_2/' + currency_type + '/'+str(k)+'.csv')
    data = data[['T', 'm', 'p']]

    temp_ask = []

    temp_bid = []

    LOB_new = []
    for i in data.index:

      if data['m'][i] == True and len(temp_bid) < 10:
        temp_bid += [data['p'][i]]
        if len(temp_bid) == 1:
          time_1 = data['T'][i]

      if data['m'][i] == False and len(temp_ask) < 10:
        temp_ask += [data['p'][i]]
        if len(temp_ask) == 1:
          time_2 = data['T'][i]
      if len(temp_bid) == 10 and len(temp_ask) == 10:
        if time_1 < time_2:
          time = time_1
        else:
          time = time_2
        LOB_new += [[time, sorted(temp_bid), sorted(temp_ask, reverse = True)]]
        temp_ask = []
        temp_bid = []
    LOB += LOB_new
    print(len(LOB))
  LOB = np.array(LOB)
  np.save('/content/drive/MyDrive/Sequential_datasets_2/' + currency_type + '/LOB_'+str(n+1), LOB)


27095
68227
107193
170467
261910
362093
80132
116064
163313
213108
259891
285003


In [4]:
# Datasets are too massive, devide them into two files
LOB_1 = np.load('/content/drive/MyDrive/Sequential_datasets_2/' + currency_type + '/LOB_1.npy', allow_pickle= True)
LOB_2 = np.load('/content/drive/MyDrive/Sequential_datasets_2/' + currency_type + '/LOB_2.npy', allow_pickle= True)

In [5]:
LOB = np.concatenate((LOB_1, LOB_2), axis = 0)

In [6]:
### Code for the final form of datasets, with times, inter-event times and types of events
new_array = []
for i in LOB:
  new_array += [[i[0], np.mean([i[1][0], i[2][0]])]]
new_array = np.array(new_array)


T = new_array[:-1,:1]
p = new_array[:,1:]

m_minus = new_array[0:-2,1:]
m_plus = new_array[2:,1:]
l_t = (m_plus - m_minus) / m_minus
inter_event = T[1:] - T[:-1]
since_last_event = np.cumsum(inter_event).reshape((-1, 1))
#price_change = p[1:] - p[:-1]

label = []
for i in l_t:
  if i > 0:
    label += [1]
  else:
    label += [0]
label = np.array(label).reshape(-1, 1)
dataset = np.concatenate((inter_event, since_last_event, label), axis = 1)

In [7]:
np.save('/content/drive/MyDrive/Sequential_datasets/Smoothed_datasets/smoothed_dataset_' + currency_type, dataset)

In [8]:
dataset = np.load('/content/drive/MyDrive/Sequential_datasets/Smoothed_datasets/smoothed_dataset_' + currency_type + '.npy', allow_pickle= True)

In [9]:
### Deviding datasets into sequences, 3000 rows in each
n_batches = int(dataset.shape[0]/3000)
batched = dataset[:n_batches*3000].reshape(n_batches, 3000, 3)

### Filtering the anomal data
times = batched[:, :, 0]
Q1 = np.percentile(times, 25)
Q3 = np.percentile(times, 75)
IQR = Q3 - Q1

new_array = np.array([]).reshape(-1, 3000, 3)
for batch in batched:
  if all(Q1 - 10*IQR <= batch[:, 0]) and all(batch[:, 0] <= Q3 + 10*IQR):
      new_array = np.concatenate((new_array, batch.reshape(-1, 3000, 3)), axis = 0)

In [10]:
dataset

array([[9.39680000e+04, 9.39680000e+04, 0.00000000e+00],
       [3.53231000e+05, 4.47199000e+05, 0.00000000e+00],
       [3.91391000e+05, 8.38590000e+05, 1.00000000e+00],
       ...,
       [1.28789000e+05, 3.15354069e+10, 0.00000000e+00],
       [3.68710000e+04, 3.15354437e+10, 1.00000000e+00],
       [4.24720000e+04, 3.15354862e+10, 1.00000000e+00]])

In [11]:
dataset.shape[0]/3000

215.698

In [12]:
batched.shape

(215, 3000, 3)

In [13]:
new_array.shape

(161, 3000, 3)

In [14]:
new_array[:,:,0].max()

570992.0

In [15]:
np.save('/content/drive/MyDrive/Sequential_datasets/Smoothed_datasets/batched_filtered_smoothed_dataset_' + currency_type, new_array)